<a href="https://colab.research.google.com/github/jwengr/KoDeBERTa/blob/main/lit_deberta_v3_colab_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install --quiet jupyter-tensorboard transformers datasets pytorch-lightning cloud-tpu-client==0.10 torch==2.0.0 torchvision==0.15.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.9/162.9 MB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.5 MB/s eta 

In [ ]:
cd /content/drive/MyDrive/KoDeBERTa/KoDeBERTa

/content/drive/MyDrive/KoDeBERTa/KoDeBERTa


In [ ]:
import os
import random
import numpy as np
import pytorch_lightning as pl

from datasets import load_dataset
from tokenizers import Tokenizer
from torch.utils.data import DataLoader
from pytorch_lightning.profilers import XLAProfiler
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from Model.DebertaV3.DebertaV3 import LitDebertaV3ForPretraining
from Data.DataCollator import DataCollatorForHFUnigramSpanMLM

In [ ]:
seed = 0
data_path = '/content/drive/MyDrive/KoDeBERTa/data/mecab.txt'
model_name = 'microsoft/deberta-v3-xsmall'
tokenizer_path = '/content/drive/MyDrive/KoDeBERTa/tokenizers/mecab-hf-unigram-880M-128k.json'
mask_token = '[MASK]'
pad_token = '[PAD]'
lr = 1e-4
current_step=0
num_training_steps = 512000000
num_warmup_steps = 40960000
batch_size = 16
log_per_steps = 50
log_dir = '/content/drive/MyDrive/KoDeBERTa/logs'
save_per_steps = 100 # 25600000
save_dir = '/content/drive/MyDrive/KoDeBERTa/models'
max_length = 512
mask_prob = 0.15
pretrained_model_path = None
gradient_checkpointing = True

In [ ]:
pl.seed_everything(seed)

INFO:lightning_fabric.utilities.seed:Global seed set to 0


0

In [ ]:
tokenizer = Tokenizer.from_file(tokenizer_path)
mask_id = tokenizer.get_vocab()[mask_token]
pad_id = tokenizer.get_vocab()[pad_token]

In [ ]:
ds = load_dataset(
    "text",
    data_files={"train": data_path},
    streaming=False,
)['train']
dl = DataLoader(ds, batch_size=batch_size, collate_fn=DataCollatorForHFUnigramSpanMLM(tokenizer, truncation_argument={'max_length':max_length}, mask_prob=mask_prob, from_hf_datasets=True), drop_last=True)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
debertav3_pretrainer = LitDebertaV3ForPretraining(
    model_name=model_name,
    mask_id=mask_id,
    pad_id=pad_id,
    lr=lr,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps,
    gradient_checkpointing=gradient_checkpointing,
    current_step=current_step,
)

In [ ]:
logger = TensorBoardLogger(log_dir, name="LitDebertaV3ForPretrainingDataCollatorForHFUnigramSpanMLM")

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath=save_dir,
    filename='LitDebertaV3ForPretrainingDataCollatorForHFUnigramSpanMLM-current_step={current_step:d}-Loss_G={Loss_G:.2f}-Loss_D={Loss_D:.2f}',
    every_n_train_steps=save_per_steps,
)

In [ ]:
trainer = pl.Trainer(
    accelerator='tpu',
    precision='16-mixed',
    max_steps=num_training_steps - current_step,
    logger=logger,
    devices=8,
    log_every_n_steps=100,
    callbacks=[checkpoint_callback],
)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:488: UserWarning: You passed `Trainer(accelerator='tpu', precision='16-mixed')` but AMP with fp16 is not supported on TPUs. Using `precision='bf16-mixed'` instead.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: True, using: 8 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(debertav3_pretrainer,dl)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /content/drive/MyDrive/KoDeBERTa/models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type                            | Params
------------------------------------------------------------------
0 | generator     | DebertaV2ForMaskedLM            | 60.3 M
1 | discriminator | DebertaV2ForTokenClassification | 70.7 M
------------------------------------------------------------------
130 M     Trainable params
0         Non-trainable params
130 M     Total params
523.980   Total estimated model params size (MB)
Exception in device=TPU:0: TPUs do not currently support IterableDataset objects, the dataset must implement `__len__`. HINT: You can mock the length on your dataset to bypass this error.Exception in device=TPU:7: TPUs do not currently 

ProcessExitedException: ignored

In [ ]:
16 0854 33.42

512